In [0]:
from pyspark.sql.functions import col,when
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
import requests
from pyspark.sql.functions import when, col, array, concat_ws
import pandas as pd


In [0]:
api_key = "408a589f7f2d505ff09be001d178006b" 
base_url = "http://api.openweathermap.org/data/2.5/weather"
city = "Seoul"
url = f"{base_url}?q={city}&appid={api_key}"
response = requests.get(url)
print(response.json())

{'coord': {'lon': 126.9778, 'lat': 37.5683}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 272.81, 'feels_like': 272.81, 'temp_min': 272.81, 'temp_max': 272.81, 'pressure': 1023, 'humidity': 39, 'sea_level': 1023, 'grnd_level': 1013}, 'visibility': 10000, 'wind': {'speed': 1.04, 'deg': 171, 'gust': 0.98}, 'clouds': {'all': 0}, 'dt': 1735661644, 'sys': {'country': 'KR', 'sunrise': 1735685212, 'sunset': 1735719833}, 'timezone': 32400, 'id': 1835848, 'name': 'Seoul', 'cod': 200}


In [0]:

Weather_list1 = []
# Your OpenWeather API key
api_key = "408a589f7f2d505ff09be001d178006b"  # Replace with your actual API key

# List of cities to get weather for
cities = ["London", "New York", "Tokyo", "Mumbai", "Sydney", "Singapore", "Seoul", "Beijing", "Cape Town","Moscow", "Paris", "Venice", "Berlin", "Ottawa", "Canberra", "Brasília", "Madrid", "Rome", "Bangkok","Nairobi", "Buenos Aires", "Wellington", "Oslo", "Cairo", "Helsinki","Abu Dhabi"]


# Base URL for the OpenWeather API
base_url = "http://api.openweathermap.org/data/2.5/weather"

# Function to fetch weather for a single city
def get_weather(city, api_key):
    try:
        # Construct the request URL
        url = f"{base_url}?q={city}&appid={api_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            # Extract useful information
            weather_info = [
                data["name"],
                round(data["main"]["temp"] - 273.15, 2),  # Convert Kelvin to Celsius
                data["weather"][0]["description"],
                data["main"]["humidity"],
                data["wind"]["speed"],
                round(data["main"]["temp_min"] - 273.15, 2),
                round(data["main"]["temp_max"] - 273.15, 2),
                data["main"]["sea_level"]
                
                
            ]
            return weather_info
        else:
            return {"city": city, "error": f"Error {response.status_code}: {response.text}"}
    except Exception as e:
        return {"city": city, "error": str(e)}

# Fetch weather for all cities
for city in cities:
    weather = get_weather(city, api_key)
    Weather_list1.append(weather)


print(Weather_list1)

df_new = pd.DataFrame(Weather_list1, columns=['City', 'temperature', 'weather', 'humidity', 'wind speed', 'Min_temperature', 'Max_temperature', 'sea_level'])

df_2 = df_new.copy()
   
# Normalize columns directly using a loop
columns_to_normalize = ['temperature', 'humidity', 'wind speed']
normalized_columns = []

# Loop through each column to normalize and store normalized column names
for column in columns_to_normalize:
    min_val = df_new[column].min()
    max_val = df_new[column].max()
    normalized_col_name = f"{column}_normalized"
    df_new[normalized_col_name] = (df_new[column] - min_val) / (max_val - min_val)
    normalized_columns.append(normalized_col_name)

# Calculate Comfort Index using a loop
comfort_index = []
for _, row in df_new.iterrows():
    temperature_score = 0.4 * (1 - abs(row['temperature_normalized'] - 0.5))
    humidity_score = 0.3 * (1 - abs(row['humidity_normalized'] - 0.5))
    wind_speed_score = 0.2 * (1 - row['wind speed_normalized'])
    weather_condition_score = 0.1 if "clear" in row['weather'].lower() else 0.05
    comfort_index.append(
        temperature_score + humidity_score + wind_speed_score + weather_condition_score
    )

# Assign the Comfort Index to the DataFrame
df_new['Comfort_Index'] = comfort_index

df_new_final = df_new[['City','Comfort_Index']].sort_values(by='Comfort_Index',ascending=False)




[['London', 6.8, 'heavy intensity rain', 90, 7.2, 5.9, 7.95, 1006], ['New York', 8.24, 'mist', 88, 5.66, 6.88, 8.98, 1000], ['Tokyo', 6.49, 'few clouds', 43, 3.09, 5.88, 8.08, 1017], ['Mumbai', 27.99, 'haze', 51, 2.57, 27.99, 27.99, 1013], ['Sydney', 23.43, 'overcast clouds', 82, 8.23, 21.79, 24.28, 1014], ['Singapore', 26.75, 'broken clouds', 83, 4.63, 26.61, 26.97, 1011], ['Seoul', -0.24, 'clear sky', 80, 2.57, -1.22, -0.24, 1024], ['Beijing', -3.06, 'broken clouds', 32, 2.02, -3.06, -3.06, 1029], ['Cape Town', 24.7, 'clear sky', 47, 7.72, 23.84, 26.72, 1013], ['Moscow', -0.42, 'light snow', 98, 6.31, -1.76, -0.35, 1000], ['Paris', 7.4, 'overcast clouds', 88, 8.23, 6.32, 8.02, 1018], ['Venice', 19.71, 'clear sky', 94, 0.45, 18.35, 19.99, 1020], ['Berlin', 7.0, 'scattered clouds', 77, 11.62, 6.66, 7.77, 1008], ['Ottawa', 2.01, 'light rain', 89, 5.66, 0.91, 2.77, 1001], ['Canberra', 17.09, 'broken clouds', 82, 3.6, 16.38, 18.07, 1016], ['Brasília', 22.51, 'thunderstorm', 83, 2.57, 22.0

In [0]:
df_new_final

,City,Comfort_Index
16,Madrid,0.866170
17,Rome,0.816110
11,Venice,0.775101
18,Bangkok,0.760388
14,Canberra,0.756986
6,Seoul,0.732631
21,Wellington,0.724986
2,Tokyo,0.722922
23,Cairo,0.715286
15,Brasília,0.704887


In [0]:
df_2

,City,temperature,weather,humidity,wind speed,Min_temperature,Max_temperature,sea_level
0,London,11.87,broken clouds,79,6.69,10.95,12.30,1017
1,New York,10.27,clear sky,54,2.06,8.94,11.27,1013
2,Tokyo,6.26,clear sky,41,6.69,5.33,6.97,1011
3,Mumbai,27.99,smoke,57,2.06,27.99,27.99,1015
4,Sydney,21.33,overcast clouds,81,2.57,19.61,22.06,1014
5,Singapore,25.68,broken clouds,86,3.09,25.50,25.97,1011
6,Seoul,-6.24,clear sky,86,0.51,-6.24,-6.24,1023
7,Beijing,-6.06,clear sky,31,1.49,-6.06,-6.06,1022
8,Cape Town,22.49,few clouds,59,5.14,21.60,23.48,1015
9,Moscow,0.16,broken clouds,89,5.38,-1.71,0.21,1007


In [0]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Example DataFrame (replace this with your full dataset)
data = [
    ['London', 11.25, 'broken clouds', 82, 6.17, 10.28, 12.3, 1018],
    ['Wellington', 13.81, 'broken clouds', 82, 10.80, 13.81, 13.81, 1005],
    ['Tokyo', 6.83, 'clear sky', 53, 9.77, 5.88, 7.79, 1009],
    ['Mumbai', 27.99, 'smoke', 54, 1.54, 27.99, 27.99, 1014],
    ['Sydney', 21.51, 'clear sky', 79, 4.12, 20.4, 22.61, 1015],
]

columns = ['City', 'Temperature', 'Weather Condition', 'Humidity', 'Wind Speed', 
           'Min Temp', 'Max Temp', 'Sea Level Pressure']

df = pd.DataFrame(data, columns=columns)

# Scale the numeric columns to a range of 0–1
scaler = MinMaxScaler()
df[['Temperature', 'Humidity', 'Wind Speed']] = scaler.fit_transform(
    df[['Temperature', 'Humidity', 'Wind Speed']]
)

# Assign weights to each parameter
def calculate_comfort_index(row):
    temperature_score = 0.4 * (1 - abs(row['Temperature'] - 0.5))  # Penalize for deviation from 0.5 (ideal temp range)
    humidity_score = 0.3 * (1 - abs(row['Humidity'] - 0.5))  # Penalize for deviation from 0.5 (ideal humidity)
    wind_speed_score = 0.2 * (1 - row['Wind Speed'])  # Lower wind speeds are better
    weather_condition_score = 0.1 if "clear" in row['Weather Condition'] else 0.05  # Favor clear skies
    return temperature_score + humidity_score + wind_speed_score + weather_condition_score

# Add Comfort Index column
df['Comfort_Index'] = df.apply(calculate_comfort_index, axis=1)

# Rank cities by Comfort Index
df = df[['City','Comfort_Index']].sort_values(by='Comfort_Index', ascending=False)




In [0]:
print(df[['City', 'Comfort Index']])

         City  Comfort Index
4      Sydney       0.747806
3      Mumbai       0.610345
0      London       0.583554
1  Wellington       0.531947
2       Tokyo       0.472246


In [0]:
df

,City,Temperature,Weather Condition,Humidity,Wind Speed,Min Temp,Max Temp,Sea Level Pressure,Comfort Index
4,Sydney,0.693762,clear sky,0.896552,0.278618,20.40,22.61,1015,0.747806
3,Mumbai,1.000000,smoke,0.034483,0.000000,27.99,27.99,1014,0.610345
0,London,0.208885,broken clouds,1.000000,0.500000,10.28,12.30,1018,0.583554
1,Wellington,0.329868,broken clouds,1.000000,1.000000,13.81,13.81,1005,0.531947
2,Tokyo,0.000000,clear sky,0.000000,0.888769,5.88,7.79,1009,0.472246


In [0]:
%fs ls 'dbfs:/FileStore/tables/fooddelivery/food_deliveryDelta.csv'

path,name,size,modificationTime
dbfs:/FileStore/tables/fooddelivery/food_deliveryDelta.csv/_delta_log/,_delta_log/,0,0
dbfs:/FileStore/tables/fooddelivery/food_deliveryDelta.csv/part-00000-46de1945-f637-4db5-b3b7-83b27bbe0201-c000.snappy.parquet,part-00000-46de1945-f637-4db5-b3b7-83b27bbe0201-c000.snappy.parquet,238778,1702029120000


In [0]:
df = spark.read.json("dbfs:/FileStore/tables/weather_alerts.json")

In [0]:
display(df)

Alert,city,humidity,max_temperature,min_temperature,sea_level,temperature,weather,wind_speed
Humidity Warning,Moscow,91,1.32,-1.76,1004,1.17,overcast clouds,4.9
Humidity Warning,Venice,95,21.14,20.56,1015,20.97,clear sky,0.45
